In [2]:
import pandas as pd

In [7]:
xls = pd.ExcelFile('2001-continue_POSCO.xlsx')
print(xls.sheet_names)

['2001-2020', '2021-2025']


In [8]:
df1 = pd.read_excel('2001-continue_POSCO.xlsx', sheet_name='2001-2020')
df2 = pd.read_excel('2001-continue_POSCO.xlsx', sheet_name='2021-2025')

In [11]:
df1.info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2088 entries, 0 to 2087
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   과제상태     2088 non-null   object        
 1   총괄과제번호   2088 non-null   float64       
 2   연구과제명    2088 non-null   object        
 3   당해시작일    2088 non-null   datetime64[ns]
 4   당해종료일    2088 non-null   datetime64[ns]
 5   과제구분     2088 non-null   object        
 6   사업코드명    2088 non-null   object        
 7   시행기관명    2088 non-null   object        
 8   연구책임자    2088 non-null   int64         
 9   직위       2088 non-null   object        
 10  사원하위그룹   2088 non-null   object        
 11  연구책임자명   2088 non-null   object        
 12  총연구비     2088 non-null   int64         
 13  직접비      2088 non-null   int64         
 14  위탁연구비    2088 non-null   int64         
 15  간접비      2088 non-null   int64         
 16  실행예산     2088 non-null   int64         
 17  입금액      2088 non-null   int64   

In [ ]:
# 원본 엑셀 2개의 시트를 불러와서 1개의 데이터프레임으로 합치기

df = pd.concat([df1, df2], ignore_index=True)

In [14]:
# 열 이름 정리 (공백 제거)

df.columns = df.columns.str.strip()

In [15]:
df.columns

Index(['과제상태', '총괄과제번호', '연구과제명', '당해시작일', '당해종료일', '과제구분', '사업코드명', '시행기관명',
       '연구책임자', '직위', '사원하위그룹', '연구책임자명', '총연구비', '직접비', '위탁연구비', '간접비',
       '실행예산', '입금액', '집행금액', '잔    액', '직접비집행비율'],
      dtype='object')

In [19]:
#  열 이름 정리 (공백 제거 및 특수문자 제거)
df.columns = df.columns.str.replace(r"[^\w\s]", "", regex=True) 
df.columns = df.columns.str.replace(r"\s+", "", regex=True)

In [17]:
print(df.columns.tolist())

['과제상태', '총괄과제번호', '연구과제명', '당해시작일', '당해종료일', '과제구분', '사업코드명', '시행기관명', '연구책임자', '직위', '사원하위그룹', '연구책임자명', '총연구비', '직접비', '위탁연구비', '간접비', '실행예산', '입금액', '집행금액', '잔액', '직접비집행비율']


In [21]:
from langchain_core.documents import Document

# 각 행을 Document로 변환
docs = []
for i, row in df.iterrows():
    content = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    docs.append(Document(page_content=content, metadata={"row": i}))


In [22]:
# 첫 번째 Document 출력

print(docs[0])

page_content='과제상태: 마감
총괄과제번호: 2.000371
연구과제명: COKE성상 결정요인에 대한 실증시험 연구
당해시작일: 2001-08-01 00:00:00
당해종료일: 2004-07-31 00:00:00
과제구분: POSCO일반과제
사업코드명: 현업과제
시행기관명: 포스코
연구책임자: 20141
직위: 대우교수
사원하위그룹: 비전임교원
연구책임자명: 이희천
총연구비: 1196753000
직접비: 732539384
위탁연구비: 0
간접비: 418853616
실행예산: 0
입금액: 1196753000
집행금액: 0
잔액: 0
직접비집행비율: 0' metadata={'row': 0}


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)


In [25]:
# 임베딩

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BM-K/KoSimCSE-roberta")


c:\Users\LG\Desktop\project\POSCO-Holdings\new_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.
c:\Users\LG\Desktop\project\POSCO-Holdings\new_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LG\.cache\huggingface\hub\models--BM-K--KoSimCSE-roberta. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To 

In [26]:
def get_embedding(text: str):
    return model.encode(text, convert_to_numpy=True)

# 임베딩 테스트
embedding = get_embedding("이 과제는 포스코와 산학협력으로 진행된다.")
print(embedding.shape)  # (768,)



(768,)


In [27]:
# 예: langchain Document 리스트를 가정
embeddings = [get_embedding(doc.page_content) for doc in docs]


In [29]:
from langchain.vectorstores import Chroma

db = Chroma.from_embeddings(
    texts=docs,
    embeddings=embeddings,
    persist_directory="./my_chroma_db"  # ← 이 폴더에 로컬 저장
)
db.persist()  # 저장 명령


AttributeError: type object 'Chroma' has no attribute 'from_embeddings'